# Descriptive figures

In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
# paths
data_dir = os.path.abspath('../data')
fig_dir = os.path.abspath('../figures')

In [ ]:
# plt.style.use('seaborn-v0_8-dark')
# plt.rcParams["font.family"] = 'times new roman'

# plot formatting
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# grid lines
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.axis'] = 'both'

plt.rcParams['grid.alpha'] = 0.5
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['hatch.linewidth'] = 1.5

dcolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

pd.set_option('future.no_silent_downcasting', True)

# orders of magnitude
M = 1_000_000
B = 1_000_000_000
T = 1_000_000_000_000

# Canada's trade partners

In [ ]:
# trade by trading partners
src = os.path.join(data_dir, 'CA-part.csv')
tp = pd.read_csv(src, index_col=[0, 1, 2])

In [ ]:
# European countries

# European Union (EU)
# The EU is a single-market trade union, and sets trade policies for all member countries
EU = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',
    'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
    'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
    'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
    'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden'
    ]

# European Economic Area (EEA)
# Free internal trade with EEA
# (though Canada has no trade with Liechtenstein)
EEA = ['Iceland', 'Liechtenstein', 'Norway'] + EU

# EAA plus UK and Switzerland (EEA+)
# EEA countries plus Switzerland: https://www.gov.uk/eu-eea
EEA_plus = EEA + ['United Kingdom', 'Switzerland']

In [ ]:
# trade series with countries across time
cous = tp.pivot_table(index=['year', 'cou'], values=tp.columns, aggfunc='sum')
cous = cous.reset_index()
cous = cous.sort_values(['year', 'tradebalance'])

cous['cou'] = cous['cou'].str.split(' (', regex=False).str[0]

# make imports negative
cous['imports'] = -cous['imports']

# label EU and EEA countries
cous['eu'] = cous['cou'].isin(EU)
cous['eea'] = cous['cou'].isin(EEA)
cous['eea+'] = cous['cou'].isin(EEA_plus)


# aggregate EU (a single-market partner) and EEA+ countries
areas = [('eu', 'European Union'), ('eea+', 'EEA+ (West Europe)')]

for abbrev, name in areas:
    
    area = cous.pivot_table(index=['year', abbrev], values=tp.columns, aggfunc='sum')
    area = area.reset_index()
    area = area[area[abbrev]]
    area = area.rename(columns={abbrev : 'cou'})
    area['cou'] = name
    area[abbrev] = True

    cous = pd.concat([cous, area])
    

# remove individual EU countries after aggregating
cond = cous['eu'] & (cous['cou']!='European Union')
cous = cous[~cond].reset_index(drop=True)

# cous = cous.drop(columns=['eu', 'eea', 'eea+'])

# calculate shares
sdfs = []
cols = tp.columns
share_cols = [(c+'share').replace('ss', 's') for c in cols]

for year, sdf in cous.groupby('year'):
    
    cond = sdf['cou'] == 'Total All Countries'
    sdf[share_cols] = sdf[cols] / sdf.loc[cond, cols].iloc[0]

    sdfs.append(sdf)

cous = pd.concat(sdfs)

In [ ]:
def set_dark_mode(dark=True):
    '''
    Set dark mode for matplotlib plots.
    '''
    if dark:

        _background_color = '#1B1B1E'
        _edge_color = 'lightgray'

    else:

        _background_color = 'white'
        _edge_color = 'dimgrey'

    
    # set background colors
    keys = ['axes.facecolor', 'figure.facecolor']
    for key in keys:
        plt.rcParams[key] = _background_color

    # set edge colors
    keys = ['text.color', 'axes.labelcolor', 'xtick.color', 'ytick.color', 'axes.edgecolor', 'grid.color']
    for key in keys:
        plt.rcParams[key] = _edge_color

    
    return _background_color, _edge_color

_background_color, _edge_color = set_dark_mode(False)

In [ ]:
# number of top
top = 10
year = 2024

# trade in latest year
cond = cous['year'] == year
df = cous[cond]

data_dict = {}

# top gross import partners
title, col = f'Canada\'s Top Import Partners ({year})', 'imports'
data = df.sort_values(col)
data_dict[title] = {'data' : data, 'col' : col}

# top gross export partners
title, col = f'Canada\'s Top Export Partners ({year})', 'exports'
data = df.sort_values(col, ascending=False)
data_dict[title] = {'data' : data, 'col' : col}


for dark in (True, False):
    
    # set dark mode
    _background_color, _edge_color = set_dark_mode(dark)
    
    for title, params in data_dict.items():
        
        data = params['data']
        col = params['col']
        
        # add 2 for all countries and EEA+
        nhead = top if 'net' in title.lower() else top + 2
        sdf = data.head(nhead)

        others = data.iloc[nhead:]
        nothers = len(others)
        others = others.sum(axis=0).to_frame().T
        others[['year', 'cou']] = 2024, f'Total {nothers} other trade partners'
        sdf = pd.concat([sdf, others]).reset_index(drop=True)

        x, y = sdf['cou'], sdf[col] / B

        fig, ax = plt.subplots(figsize=(8, 5))
        
        # formatting
        kwargs = {
            'color' : 'tomato' if 'export' in title.lower() else 'royalblue',
            'fill' : False if 'net' in title.lower() else True,
            'linewidth' : 3
        }
        kwargs['edgecolor'] = kwargs['color'] if 'net' in title.lower() else None

        # plot bar
        bars = ax.barh(x, y, **kwargs)

        # ax.set_title(title) # remove title for article
        ax.set_xlabel(f'{col.title()} in Billions CAD')
        
        # reverse negative sign for imports
        if 'import' in title.lower():
            xlim = ax.get_xlim()    
            xticks = ax.get_xticks()
            ax.set_xticks(xticks, -xticks.astype(int))
            ax.set_xlim(xlim)

        
        # # add source and title
        # ax.set_title(title)
        # fig.text(1, 0, 'Data source: ISED Canada   ', ha='right', va='bottom')

        # add hatch to bar for total all countries export/import
        if 'net' not in title.lower():
            for i in (0, 2, -1):
                bars[i].set_hatch('//')
                bars[i].set_linewidth(0)
                bars[i].set_edgecolor(_background_color)

            fig.legend((bars[0], bars[1]), ('Aggregate of trade partners', 'Single-market trade partners'),
                    bbox_to_anchor=(0, 0), loc='center left', frameon=False)

            
        # invert order of countries
        fig.gca().invert_yaxis()
        
        # save figures
        n = 4 if 'net' in title.lower() else 3
        tag = '_'.join(title.split()[-n:-1])
        
        if dark:
            tag += '-dark'
        
        file = f'03-{tag}.png'.lower()

        fig.tight_layout()

        dst = os.path.join(fig_dir, file)
        plt.savefig(dst, bbox_inches='tight')

        print('Saved:', dst)


        plt.show()
        display(pd.merge(x, y, left_index=True, right_index=True).reset_index(drop=True))

In [ ]:
partners = ['United States', 'EEA+ (West Europe)', 'China']

cols = ['importshare', 'exportshare']

# aggregate other trade partners
cond1 = ~(cous['cou'].isin(partners)) # already listed partners
cond2 = ~(cous['eea+'].fillna(True).astype(bool)) # EEA+ countries
cond3 = ~cous['cou'].str.contains('total', case=False) # aggregates
cond4 = ~cous['cou'].str.contains('import', case=False) # re-imports

cond = cond1 & cond2 & cond3 & cond4

sdf = cous[cond]

others = sdf.groupby('year').sum()
others = others.reset_index()

label = f'All other\n trade partners'
others['cou'] = label
partners.append(label)

# add non-US import/export share
cond = cous['cou'] == 'United States'
non_us = 1 - (cous.loc[cond, ['year']+cols].set_index('year'))
non_us = non_us.reset_index()

label = 'Total non-US'
non_us['cou'] = label
partners.append(label)

# create final dataset
cond = cous['cou'].isin(partners)
data = pd.concat([cous[cond], others, non_us])

data = data.sort_values(['year', 'cou'])

# account
linestyles = [':', '-', '-.']

# partners
markers = ['o', '+', '*', 'H', 'x']
colors = ['royalblue', 'green', 'orange', 'gray', 'tomato']


for dark in (True, False):
    
    # set dark mode
    _background_color, _edge_color = set_dark_mode(dark)
    
    for i, col in enumerate(cols):
        
        fig, ax = plt.subplots(figsize=(8, 5))

        for j, cou in enumerate(partners):
                
                cond = data['cou'] == cou
                x, y = data.loc[cond, 'year'], data.loc[cond, col]

                ax.plot(x, y, lw=2, ls=linestyles[i],
                        marker=markers[j], fillstyle='none',
                        markersize=8, markeredgewidth=2,
                        label=cou, color=colors[j])
                
                ax.set_xlim(2000, 2024)
                ax.set_xticks(range(2000, 2025, 4))
                ax.set_xticks(range(2000, 2025), minor=True)
                
                xend, yend = x.dropna().iloc[-1], y.dropna().iloc[-1]

                va = 'bottom' if (('other' in cou) and ('export' in col)) else 'center'
                ax.text(xend, yend, f' {cou}', va=va,
                        ha='left', color=colors[j],
                        fontweight='bold'
                        )

        # set ylim and 10% yticks
        _, ymax = ax.get_ylim()
        step = 0.1
        ax.set_yticks(np.arange(0, ymax+step, step))
        ax.set_ylim(0, ymax)
        ax.yaxis.minorticks_off()

        ax.set_ylabel(f'Share of {col.replace("hare", "").title()}')
        ticks = ax.get_yticks()
        ax.set_yticks(ticks, [f'{t:.0%}' for t in ticks])
        ax.yaxis.minorticks_off()

        fig.tight_layout()
        # fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0), ncol=3, frameon=False)
        
        tag = f'03-{col}'
        if dark:
                tag += '-dark'

        dst = os.path.join(fig_dir, f'{tag}.png')
        
        fig.savefig(dst, bbox_inches='tight')
        print('Saved:', dst)
        
        plt.show()

## What do we trade?

In [ ]:
# Canada's trade by sector/product

# keep only US and world
prods = tp.reset_index()
cond = prods['cou'].isin(['United States', 'Total All Countries'])
prods = prods[cond]
prods['imports'] = -prods['imports']

# add sector HS2 codes
src = os.path.join(data_dir, 'HS2_codes.csv')
codes = pd.read_csv(src)
split = codes['code'].str.split(' - ')
codes['hs2'] = split.str[0].astype(int)

prods = pd.merge(prods, codes, on='hs2', how='left')
prods

prods = prods.pivot_table(index=['year', 'sec', 'hs2', 'code'], columns='cou', values=tp.columns, aggfunc='sum')
cols = [(b, a) for a, b in prods.columns.tolist()]
prods.columns = pd.MultiIndex.from_tuples(cols)
prods = prods[sorted(cols)]

non_us = prods['Total All Countries'] - prods['United States'] 
cols = [('Non-US', c) for c in tp.columns]
cols = pd.MultiIndex.from_tuples(cols)
prods[cols] = non_us

In [ ]:
secs = prods.pivot_table(index=['year', 'sec'], values=prods.columns, aggfunc='sum')

for year, sdy in secs.groupby('year'):
    continue

for dark in (True, False):

    # set dark mode
    _background_color, _edge_color = set_dark_mode(dark)
        
    for col in ['imports', 'exports']:

        fig, ax = plt.subplots(figsize=(10, 5))

        ascending = True if col=='imports' else False
        sdy = sdy.sort_values([('Total All Countries', col)], ascending=ascending)


        areas = ['United States', 'Non-US']
        colors = ['royalblue', 'tomato']
        hatches = ['', '///']
        left = np.zeros(len(sdy))

        for i, area in enumerate(areas):

            sdf = sdy[area].reset_index()

            sdf['rank'] = range(1, len(sdf)+1)
            split = sdf['sec'].str.split(' - ')
            sdf['sector'] = sdf['rank'].astype(str) + '. ' + split.str[-1].str.title()

            x, y = sdf['sector'], sdf[col] / B

            ax.barh(x, y, label=area,
                    color=colors[i], left=left, linewidth=0,
                    hatch=hatches[i], edgecolor=_background_color,
                    )
            
            left += y.to_numpy()


        ax.set_xlabel(f'{col.title()} in Billions CAD')
        fig.gca().invert_yaxis()

        if col == 'imports':
    
            xlim = ax.get_xlim()
            xticks = ax.get_xticks()
            ax.set_xticks(xticks, -xticks.astype(int))
            ax.set_xlim(xlim)


        fig.tight_layout()
        fig.legend(bbox_to_anchor=(0, 0), ncol=2, loc='lower left', frameon=False)

        tag = f'03-sector-{col}'
        if dark:
            tag += '-dark'
            
        dst = os.path.join(fig_dir, f'{tag}.png')
        fig.savefig(dst, bbox_inches='tight')
        print('Saved:', dst)

        plt.show()
        

In [ ]:
# stacked bar graph for US/non-US

# sectors
total = prods['Total All Countries']
secs = total.pivot_table(index=['year', 'sec'], values=tp.columns, aggfunc='sum')
secs = secs.reset_index()

for year, sdf in secs.groupby('year'):
    continue

col = 'imports'
sdf = sdf.sort_values(col, ascending=True if col=='imports' else False)

sdf['rank'] = range(1, len(sdf)+1)
split = sdf['sec'].str.split(' - ')
sdf['sector'] = sdf['rank'].astype(str) + '. ' + split.str[-1].str.title()

fig, ax = plt.subplots()

x, y = sdf['sector'], sdf[col] / B
ax.barh(x, y)

fig.gca().invert_yaxis()

plt.show()

fig, ax = plt.subplots()

for sec, sdf in secs.groupby('sec'):

    x, y = sdf['year'], sdf['exports'] / B

    ax.plot(x, y)

    ax.set_xlim(2000, 2024)
    ax.set_xticks(range(2000, 2025, 4))

    text = ' ' + (sec.split(' - ')[-1].title())
    ax.text(x.iloc[-1], y.iloc[-1], text, va='center', ha='left')

plt.show()

## Trade by Province

In [ ]:
# Breakdown trade by sector with top trader